In [1]:
import requests
import json

## 会社を生成

In [5]:
company_res = requests.post('http://localhost:3000/companies',json={'name': 'ヘルプ株式会社'})

In [6]:
assert company_res.status_code == 201

In [7]:
depart_res = requests.post('http://localhost:3000/departments',json={'name': '役員', 'company_id': company_res.json()['id']})

In [8]:
assert depart_res.status_code == 201

In [2]:
system_admin_email = 'example33@example.com'
system_admin_passwd = 'p@ssword'

## システム管理者アカウント作成

In [10]:
res = requests.post('http://localhost:3000/auth',json={
    'email': system_admin_email,
    'password': system_admin_passwd,
    'password_confirmation': system_admin_passwd
})

In [11]:
res.json()['data']['id']

1

In [12]:
system_admin_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'system_admin'
})

In [13]:
depart_res.json()['id']

1

In [14]:
company_res.json()['id']

1

In [15]:
assert system_admin_res.status_code == 201

In [16]:
res = requests.post('http://localhost:3000/auth',json={
    'email': 'hoge@example.com',
    'password': 'pass',
    'password_confirmation': 'pass'
})
admin_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'system_admin'
})

In [17]:
assert admin_res.status_code == 403

In [18]:
admin_res

<Response [403]>

## ログイン

In [3]:
system_admin_login_res = requests.post('http://localhost:3000/auth/sign_in',json={
    'email': system_admin_email,
    'password': system_admin_passwd
})

In [4]:
system_admin_login_res.json()

{'data': {'id': 1,
  'email': 'example33@example.com',
  'provider': 'email',
  'uid': 'example33@example.com',
  'allow_password_change': False,
  'name': None,
  'nickname': None,
  'image': None}}

In [5]:
system_admin_login_headers = {x:system_admin_login_res.headers[x] for x in ['client', 'access-token', 'uid']}

In [111]:
res = requests.get('http://localhost:3000/search-problem?company_id=1', headers=system_admin_login_headers)
res.json()

[{'id': 7,
  'user_id': 3,
  'genre_id': 1,
  'created_at': '2020-03-21T15:43:33.244Z',
  'updated_at': '2020-03-21T15:43:33.244Z',
  'image_url': 'https://help-api.storage.googleapis.com/help/v1/5fb336de380b37c1605a42ff1ec521ae.jpg',
  'company_id': 1},
 {'id': 8,
  'user_id': 3,
  'genre_id': 1,
  'created_at': '2020-03-21T15:50:44.226Z',
  'updated_at': '2020-03-21T15:50:44.226Z',
  'image_url': 'https://help-api.storage.googleapis.com/help/v1/a9d5608105870585967fa5406bc6d2a6.jpg',
  'company_id': 1},
 {'id': 9,
  'user_id': 1,
  'genre_id': 1,
  'created_at': '2020-03-21T17:18:04.291Z',
  'updated_at': '2020-03-21T17:18:04.291Z',
  'image_url': 'https://help-api.storage.googleapis.com/help/v1/acc225e2b36c996ee9e0e5122f7f3715.jpg',
  'company_id': 1},
 {'id': 10,
  'user_id': 1,
  'genre_id': 1,
  'created_at': '2020-03-21T17:22:01.723Z',
  'updated_at': '2020-03-21T17:22:01.723Z',
  'image_url': 'https://help-api.storage.googleapis.com/help/v1/0bb3fccea586bb451c0c0f0c836ac83c.jpg',

## 管理者ユーザー作成

In [22]:
admin_email = 'example@example.com'
admin_password = 'ex@mple'

In [23]:
res = requests.post('http://localhost:3000/auth',json={
    'email': admin_email,
    'password': admin_password,
    'password_confirmation': admin_password
})

In [24]:
admin_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'admin'
},headers=system_admin_login_headers)

In [25]:
show_admin_infos_res = requests.get(
    'http://localhost:3000/user_infos/%s' % admin_res.json()['id'],
)
assert show_admin_infos_res.status_code == 403

In [26]:
show_admin_infos_res = requests.get(
    'http://localhost:3000/user_infos/%s' % admin_res.json()['id'],
    headers=system_admin_login_headers
)
assert show_admin_infos_res.status_code == 200

In [27]:
assert admin_res.status_code == 201

In [28]:
admin_login_res = requests.post('http://localhost:3000/auth/sign_in',json={
    'email': admin_email,
    'password': admin_password,
    'password_confirmation': admin_password,
})

In [29]:
admin_login_headers = {x:admin_login_res.headers[x] for x in ['client', 'access-token', 'uid']}

## 管理者ユーザーが同じ会社のユーザーを作成する

In [10]:
normal_email = "example11@example.com"
normal_pass = "example11@"

In [36]:
res = requests.post('http://localhost:3000/auth',json={
    'email': normal_email,
    'password': normal_pass,
    'password_confirmation': normal_pass
})

In [37]:
normal_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'admin'
},headers=admin_login_headers)

In [11]:
normal_login_res = requests.post('http://localhost:3000/auth/sign_in',json={
    'email': normal_email,
    'password': normal_pass
})

In [12]:
normal_login_headers = {x:normal_login_res.headers[x] for x in ['client', 'access-token', 'uid']}

In [14]:
assert normal_login_res.status_code == 200

## 管理者ユーザーが違う会社のユーザーを作成すると失敗する

In [37]:
other_company_res = requests.post('http://localhost:3000/companies',json={'name': 'アザー株式会社'})

In [38]:
res = requests.post('http://localhost:3000/auth',json={
    'email': "example13@example.com",
    'password': "example13@",
    'password_confirmation': "example13@"
})

In [40]:
other_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': other_company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'admin'
},headers=admin_login_headers)

In [41]:
assert other_res.status_code == 403

## 管理者ユーザーがユーザーの会社を変えると失敗する

In [42]:
other_res = requests.put('http://localhost:3000/user_infos/%s' % normal_res.json()['id'],json={
    'department_id': depart_res.json()['id'],
    'company_id': 5,
    'user_id': normal_res.json()['user_id'],
    'role': 'admin'
},headers=admin_login_headers)

In [43]:
other_res

<Response [403]>

In [44]:
assert other_res.status_code == 403

In [45]:
normal_res.json()

{'id': 3,
 'company_id': 1,
 'department_id': 1,
 'user_id': 3,
 'created_at': '2020-03-21T10:46:30.248Z',
 'updated_at': '2020-03-21T10:46:30.248Z',
 'role': 'admin',
 'name': None}

## 管理者ユーザーが同じ会社のユーザーの部門を変える

In [46]:
another_depart_res = requests.post('http://localhost:3000/departments',json={'name': 'アナザー', 'company_id': company_res.json()['id']})

In [47]:
same_comp_res = requests.put('http://localhost:3000/user_infos/%s' % normal_res.json()['id'],json={
    'department_id': another_depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': normal_res.json()['user_id'],
    'role': 'admin'
},headers=admin_login_headers)

In [48]:
assert same_comp_res.status_code == 200

In [65]:
res = requests.post('http://localhost:3000/questions',
json={
    'answer_headline': '概要',
    'enquete_headline': '20文字程度でどんな悩みか教えてください'
},headers=system_admin_login_headers)
assert res.status_code == 201

In [66]:
res = requests.post('http://localhost:3000/questions',
json={
    'answer_headline': '詳細',
    'enquete_headline': 'もっと詳しく教えてください'
},headers=system_admin_login_headers)
assert res.status_code == 201

In [54]:
res = requests.post('http://localhost:3000/genres',
json={
    'name': '人間関係',
},headers=system_admin_login_headers)

In [55]:
res = requests.post('http://localhost:3000/genres',
json={
    'name': '仕事',
},headers=system_admin_login_headers)

In [56]:
res = requests.post('http://localhost:3000/genres',
json={
    'name': '健康',
},headers=system_admin_login_headers)

In [57]:
#TODO: システム管理者以外はgenreを生成できない

In [101]:
res = requests.post('http://localhost:3000/problems',
json={
    'user_id': 3,
    'genre_id': 1,
    'base64_image': base64_str,
    'company_id': company_res.json()['id']
},headers=normal_login_headers)

In [100]:
res = requests.post('http://localhost:3000/problems',
json = {
    "user_id": 1,
    "genre_id":1,
    "base64_image": "PHN2ZyBkYXRhLXYtNDBhMzc0YjU9IiIgaWQ9ImNhbnZhcyIgd2lkdGg9IjY0MCIgaGVpZ2h0PSIzNjAiIHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8yMDAwL3N2ZyI+PGcgZGF0YS12LTQwYTM3NGI1PSIiPjx0aXRsZSBkYXRhLXYtNDBhMzc0YjU9IiI+YmFja2dyb3VuZDwvdGl0bGU+IDxyZWN0IGRhdGEtdi00MGEzNzRiNT0iIiBpZD0iY2FudmFzX2JhY2tncm91bmQiIGZpbGw9IiNmNDUzNTMiIGhlaWdodD0iMzYyIiB3aWR0aD0iNjQyIiB5PSItMSIgeD0iLTEiLz4gPGcgZGF0YS12LTQwYTM3NGI1PSIiIGlkPSJjYW52YXNHcmlkIiBkaXNwbGF5PSJub25lIiBvdmVyZmxvdz0idmlzaWJsZSIgeT0iMCIgeD0iMCIgaGVpZ2h0PSIxMDAlIiB3aWR0aD0iMTAwJSI+PHJlY3QgZGF0YS12LTQwYTM3NGI1PSIiIGZpbGw9InVybCgjZ3JpZHBhdHRlcm4pIiBzdHJva2Utd2lkdGg9IjAiIHk9IjAiIHg9IjAiIGhlaWdodD0iMTAwJSIgd2lkdGg9IjEwMCUiLz48L2c+PC9nPiA8ZyBkYXRhLXYtNDBhMzc0YjU9IiI+PHRpdGxlIGRhdGEtdi00MGEzNzRiNT0iIj5MYXllciAxPC90aXRsZT4gPHRleHQgZGF0YS12LTQwYTM3NGI1PSIiIGlkPSJzdmdfNCIgZm9udC1zdHlsZT0ibm9ybWFsIiBmb250LXdlaWdodD0iYm9sZCIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSIgdGV4dC1hbmNob3I9InN0YXJ0IiBmb250LWZhbWlseT0iSGVsdmV0aWNhLCBBcmlhbCwgc2Fucy1zZXJpZiIgZm9udC1zaXplPSI0MCIgeT0iNjQuNDUzMTI1IiB4PSIzMiIgc3Ryb2tlLXdpZHRoPSIwIiBzdHJva2U9IiMwMDAiIGZpbGw9IiNmZmZmZmYiPmdlZzwvdGV4dD4gPHRleHQgZGF0YS12LTQwYTM3NGI1PSIiIGlkPSJzdmdfNiIgc3Ryb2tlPSIjMDAwIiB0cmFuc2Zvcm09Im1hdHJpeCgzOC40ODMxNDY2Njc0ODA0NywwLDAsMSwtMjAzNzYuNzc1NjA3MTA5MDcsMCkgIiB4bWw6c3BhY2U9InByZXNlcnZlIiB0ZXh0LWFuY2hvcj0ic3RhcnQiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EsIEFyaWFsLCBzYW5zLXNlcmlmIiBmb250LXNpemU9IjIwIiB5PSIxNzMuNDUzMTI1IiB4PSI1MzIuNSIgc3Ryb2tlLXdpZHRoPSIwIiBmaWxsPSIjZmZmZmZmIi8+IDx0ZXh0IGRhdGEtdi00MGEzNzRiNT0iIiBpZD0ic3ZnXzkiIHhtbDpzcGFjZT0icHJlc2VydmUiIHRleHQtYW5jaG9yPSJzdGFydCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSwgQXJpYWwsIHNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMjIiIHk9IjEyOS40NTMxMjUiIHg9IjY0LjUiIGZpbGwtb3BhY2l0eT0ibnVsbCIgc3Ryb2tlLW9wYWNpdHk9Im51bGwiIHN0cm9rZS13aWR0aD0iMCIgc3Ryb2tlPSIjMDAwIiBmaWxsPSIjZmZmZmZmIj5nZWdlZzwvdGV4dD4gPHRleHQgZGF0YS12LTQwYTM3NGI1PSIiIGlkPSJzdmdfMTIiIHhtbDpzcGFjZT0icHJlc2VydmUiIHRleHQtYW5jaG9yPSJzdGFydCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSwgQXJpYWwsIHNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMjIiIHk9IjE3Ni40NTMxMjUiIHg9IjYzLjUiIGZpbGwtb3BhY2l0eT0ibnVsbCIgc3Ryb2tlLW9wYWNpdHk9Im51bGwiIHN0cm9rZS13aWR0aD0iMCIgc3Ryb2tlPSIjMDAwIiBmaWxsPSIjZmZmZmZmIj48L3RleHQ+IDx0ZXh0IGRhdGEtdi00MGEzNzRiNT0iIiBpZD0ic3ZnXzEzIiB4bWw6c3BhY2U9InByZXNlcnZlIiB0ZXh0LWFuY2hvcj0ic3RhcnQiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EsIEFyaWFsLCBzYW5zLXNlcmlmIiBmb250LXNpemU9IjIyIiB5PSIyMjkuNDUzMTI1IiB4PSI2MS41IiBmaWxsLW9wYWNpdHk9Im51bGwiIHN0cm9rZS1vcGFjaXR5PSJudWxsIiBzdHJva2Utd2lkdGg9IjAiIHN0cm9rZT0iIzAwMCIgZmlsbD0iI2ZmZmZmZiI+PC90ZXh0PiA8dGV4dCBkYXRhLXYtNDBhMzc0YjU9IiIgaWQ9InN2Z18xNCIgeG1sOnNwYWNlPSJwcmVzZXJ2ZSIgdGV4dC1hbmNob3I9InN0YXJ0IiBmb250LWZhbWlseT0iSGVsdmV0aWNhLCBBcmlhbCwgc2Fucy1zZXJpZiIgZm9udC1zaXplPSIyMiIgeT0iMjgzLjQ1MzEyNSIgeD0iNTkuNSIgZmlsbC1vcGFjaXR5PSJudWxsIiBzdHJva2Utb3BhY2l0eT0ibnVsbCIgc3Ryb2tlLXdpZHRoPSIwIiBzdHJva2U9IiMwMDAiIGZpbGw9IiNmZmZmZmYiPjwvdGV4dD48L2c+PC9zdmc+",
    "company_id": 1,
},headers=system_admin_login_headers)

In [101]:
res.json()

{'id': 20,
 'user_id': 1,
 'genre_id': 1,
 'created_at': '2020-03-22T03:33:17.411Z',
 'updated_at': '2020-03-22T03:33:17.411Z',
 'image_url': 'https://help-api.storage.googleapis.com/help/v1/5b6f53cd081155dce1cc9724e8e296c2.svg',
 'company_id': 1}

In [8]:
problem_id = res.json()['id']

## 自身がproblemを見るとuser_idが付与されている

In [102]:
res = requests.get('http://localhost:3000/problems/%s' % problem_id,headers=normal_login_headers)

In [103]:
assert 'user_id' in res.json()

In [104]:
res.json()

{'id': 14,
 'user_id': 1,
 'genre_id': 1,
 'created_at': '2020-03-22T01:30:34.157Z',
 'updated_at': '2020-03-22T01:30:34.157Z',
 'image_url': 'https://help-api.storage.googleapis.com/help/v1/3f4bec18fab246673b49b5180bc89829.jpg',
 'company_id': 1}

## 他の社員がproblemを見るとuser_idが付与されていない

In [24]:
res = requests.get('http://localhost:3000/problems/%s' % problem_id,headers=normal2_login_headers)

In [25]:
assert not 'user_id' in res.json()

## 管理者がproblemを見るとuser_idが付与されている

In [115]:
res = requests.get('http://localhost:3000/problems/%s' % problem_id,headers=admin_login_headers)

In [116]:
assert 'user_id' in res.json()

## 自身がanswerを検索するとuser_idが付与されている

In [98]:
res = requests.get('http://localhost:3000/search-answer?company_id=1&problem_id=1',headers=normal_login_headers)
assert res.status_code == 200
for x in res.json():
    assert 'user_id' in x

## 他の社員がanswerを検索するとuser_idが付与されていない

In [99]:
res = requests.get('http://localhost:3000/search-answer?company_id=1&problem_id=1',headers=normal2_login_headers)
assert res.status_code == 200
for x in res.json():
    assert not 'user_id' in x

In [94]:
res.json()

[{'id': 1,
  'question_id': 1,
  'user_id': 3,
  'problem_id': 1,
  'company_id': 1,
  'department_id': 1,
  'content': '体調が悪くて困っています',
  'created_at': '2020-03-21T15:08:55.782Z',
  'updated_at': '2020-03-21T15:08:55.782Z'},
 {'id': 2,
  'question_id': 2,
  'user_id': 3,
  'problem_id': 1,
  'company_id': 1,
  'department_id': 1,
  'content': '体調が悪くて困っています',
  'created_at': '2020-03-21T15:10:27.532Z',
  'updated_at': '2020-03-21T15:10:27.532Z'}]

In [59]:
#TODO: 自身の悩みしか投稿できない

In [67]:
res = requests.post('http://localhost:3000/answers',
json={
    'content': '体調が悪くて困っています',
    'department_id': 1,
    'company_id': 1,
    'user_id': normal_login_res.json()['data']['id'],
    'question_id': 1,
    'problem_id': 1
},headers=normal_login_headers)
assert res.status_code == 201

In [69]:
answer_res = requests.post('http://localhost:3000/answers',
json={
    'content': '体調が悪くて困っています',
    'department_id': 1,
    'company_id': 1,
    'user_id': normal_login_res.json()['data']['id'],
    'question_id': 2,
    'problem_id': 1
},headers=normal_login_headers)
assert answer_res.status_code == 201

## 自身の質問のユーザーを確認できる

In [21]:
res = requests.get('http://localhost:3000/answers/%s' % answer_res.json()['user_id'],headers=normal_login_headers)
assert 'user_id' in res.json().keys()

NameError: name 'answer_res' is not defined

## ログインしていないユーザーは確認できない

In [101]:
res = requests.get('http://localhost:3000/answers/%s' % answer_res.json()['user_id'])
assert res.status_code == 403

## 異なるユーザーは確認できない

In [16]:
normal2_email = "example112@example.com"
normal2_pass = "example112@"

In [108]:

res = requests.post('http://localhost:3000/auth',json={
    'email': normal2_email,
    'password': normal2_pass,
    'password_confirmation': normal2_pass
})
normal2_res = requests.post('http://localhost:3000/user_infos',json={
    'department_id': depart_res.json()['id'],
    'company_id': company_res.json()['id'],
    'user_id': res.json()['data']['id'],
    'role': 'normal'
},headers=admin_login_headers)

In [17]:
normal2_login_res = requests.post('http://localhost:3000/auth/sign_in',json={
    'email': normal2_email,
    'password': normal2_pass
})

In [18]:
assert normal2_login_res.status_code == 200
normal2_login_headers = {x:normal2_login_res.headers[x] for x in ['client', 'access-token', 'uid']}

In [19]:
res.json()['data']['id']

KeyError: 'data'

In [20]:
res = requests.get('http://localhost:3000/answers/%s' % answer_res.json()['user_id'],headers=normal2_login_headers)

NameError: name 'answer_res' is not defined

In [130]:
assert res.status_code == 200

In [140]:
assert 'id' in res.json()
assert not 'user_id' in res.json()

## 全表示の権限確認

In [89]:
res = requests.get('http://localhost:3000/answers/', headers=system_admin_login_headers)
assert res.status_code == 200
res = requests.get('http://localhost:3000/answers/', headers=admin_login_headers)
assert res.status_code == 403

## パスワード変更

In [31]:
res = requests.put('http://localhost:3000/auth/password',
headers=login_headers,
json={
    'email': 'example@example.com',
    'password': 'password1',
    'password_confirmation': 'password1'
})

## 会社を変更できないかを確認

In [46]:
res = requests.put('http://localhost:3000/user_info/%s' % ,
    json = {
        'id': 2,
        'role': 'normal',
        'department_id': 1,
        'company_id': 2,
        'user_id': 1
    }
)

In [47]:
res

<Response [404]>

In [124]:
res.json()

{'status': 403, 'message': 'Forbidden'}

In [ ]:
https://qiita.com/Masahiro_T/items/6bc49a625b437a7c2f45

In [70]:
import base64

img_file = '/Users/onosawa/Downloads/board-361516_640.jpg'
b64 = base64.encodestring(open(img_file, 'rb').read())

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: encodestring() is a deprecated alias since 3.1, use encodebytes()
  after removing the cwd from sys.path.


In [71]:
base64_str = b64.decode('utf8')

In [94]:
res = requests.post('http://localhost:3000/upload', json={'base64_image': base64_str}, headers=system_admin_login_headers)

In [95]:
res.json()

{'status': 403, 'message': 'Forbidden'}

rails g devise:install
rails g devise user
rails g scaffold department name:string company:references
rails g scaffold company name:string
rails g devise_token_auth:install User auth
rails g scaffold department name:string company:references
rails g scaffold user_info company:references department:references user:references name:string
rails g scaffold problem user:references genre:references company:references
rails g scaffold comment user:references problem:references content:string
rails g scaffold enquete
rails g scaffold question answer_headline:string enquete_headline:string
rails g scaffold enquete_question enquete:references question:references
rails g scaffold answer question:references user:references problem:references company:references department:references content:string
rails db:migrate

In [ ]:
298  gem install rails -v 5.1.6
  299  sudo gem install rails -v 5.1.6
  324  gem install rails -v 5.1.6
  329  rails new help-api
  337  rails server
  340  rails new help-api --api
  344  rails s
  345  rails g scaffold company name:string
  346  rails db:migrate
  347  rails s
  348  rails s
  351  rails s
  352  rails db:migrate:reset
  353  rails s
  354  rails g scaffold department name:string company:references
  355  rails s
  356  rails s
  357  rails db:migrate
  358  rails s
  361  rails g devise:install
  362  rails g devise user
  363  rails db:migrate
  364  rails s
  365  rails g devise:controllers users
  366  rails s
  368  rails g devise:views
  369  rails s
  372  rails s
  386  rails g scaffold company name:string
  391  rails new help-api --api
  394  rails g scaffold company name:string
  399  rails new help-api
  404  rails s
  405  rails g scaffold company:name
  406  rails g scaffold company name:string
  407  rails g scaffold deppartment name:string
  408  rails destroy scaffold department
  409  rails g scaffold deppartment name:string company:references
  411  rails g scaffold deppartment name:string company:references
  412  rails g scaffold deppartment name:string company:references
  413  rails s
  414  rails db:migrate
  415  rails s
  455  rails g scaffold company:name
  478  rails g scaffold company name:string
  479  rails g scaffold deppartment name:string company:references
  480  rails db:migrate
  481  rails g devise_token_auth:install User auth
  482  rails db:migrate
  485  rails db:migrate
  486  rails g devise_token_auth:install User auth
  490  rails g devise_token_auth:install User auth
  498  rails g devise_token_auth:install User auth
  509  rails g devise:install
  510  rails g devise_token_auth:install User auth
  520  rails s
  521  rails db:migrate
  522  rails s
  525  rails s
  529  rails g scaffold user_info company:references depertment:references user:references
  530  rails s
  531* rails db:reset
  532  rails s
  534  rails g scaffold user_info destoroy
  535  rails destroy scaffold user_info
  537  rails g scaffold company name:string
  538  rails g scaffold deppartment name:string company:references
  539  rails g scaffold user_info company:references depertment:references user:references
  541  rails db:migrate
  542  rails db:reset
  544  rails db:reset
  545  rails db:migrate
  546  rails s
  552  rails db:migrate
  554  rails s
  555  rails destroy scaffold deppartment
  556  rails g scaffold department name:string company:references
  557  rails destroy scaffold user_infos
  558  rails g scaffold user_info company:references department:references user:references
  560  rails db:migrate
  561  rails s
  565* rails g scaffold comment content:string user:references 
  566  rails destroy scaffold comment
  567  rails g scaffold problem user:references genre:references
  568  rails g scaffold comment user:references problem:references
  570  rails db:migrate
  571  rails g scaffold enquete
  572  rails g scaffold question answer_headline:string enquete_headline:string
  573  rails g scaffold enquate_question enquate:references question:references
  574  rails db:migrate
  576  rails db
  584  rails db:migrate
  586  rails d scaffold enquate_question
  587  rails g scaffold enquete_question enquete:references question:references
  589  rails db:migrate
  592  rails g scaffold answer question:references user:references problem:references company:references department:references content:string
  593  rails db:migrate

In [142]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/onosawa/Downloads/machine-learning-234808-76319dafe8ce.json'

In [143]:
import os
from google.cloud import storage
client = storage.Client()

In [146]:
bucket = client.get_bucket('help-api')

In [149]:
blob = bucket.blob('help/test.jpg')
blob.upload_from_filename(filename='/Users/onosawa/Downloads/board-361516_640.jpg')

In [150]:
for f in bucket.list_blobs():
    print(f)

<Blob: help-api, /help/test.jpg, 1584793356682331>
<Blob: help-api, help/, 1584793350794082>
<Blob: help-api, help/test.jpg, 1584793489374452>
